In [ ]:
from fuzix.sims.blocksprings import draw_world
from fuzix.sims.blocksprings import get_block_params
from fuzix.sims.blocksprings import get_spring_params
from fuzix.sims.blocksprings import render_animation
from fuzix.sims.blocksprings import set_block_params
from fuzix.sims.blocksprings import set_block_state
from fuzix.sims.blocksprings import set_spring_params
from fuzix.sims.blocksprings import simulate
from fuzix.utils import memoize, RendererWidget
from IPython.display import HTML
from ipywidgets import interact
from matplotlib import pyplot as plt, cm, animation, rc
from matplotlib.lines import Line2D
import functools
import ipywidgets
import matplotlib
import matplotlib.patches as patches
import numpy as np

In [ ]:
%matplotlib inline
rc('animation', html='html5')
#rc('animation', html='jshtml')

In [ ]:
SAMPLE_INTERVAL = 2
MAX_TIME_INDEX = 200


renderer_widget = RendererWidget()
display(renderer_widget)


def show_render(params, states, scale):
    anim = render_animation(params, states, scale, sample_interval=SAMPLE_INTERVAL)
    anim.save('anim.mp4')
    display(anim)


@interact(
    time_index=(0, MAX_TIME_INDEX - 1, 1),
    initial_x1=(-4., 4.),
    initial_x2=(-4., 4.),
    initial_y1=(-4., 4.),
    initial_y2=(-4., 4.),
    initial_z1=(-4., 4.),
    initial_z2=(-4., 4.),
    b=(0., 6., 0.05),
    k=(0.01, 8.),
    m1=(0.1, 5.),
    m2=(0.1, 5.),
    m3=(0.1, 5.),
    spring_length=(1., 3.),
    dt=(0.02, 0.1, 0.01),
    scale=(2., 15.),
)
def f(
        time_index=0,
        initial_x1=-3.,
        initial_x2=2.,
        initial_y1=0.,
        initial_y2=0.,
        initial_z1=3.,
        initial_z2=0.,
        b=0.2,
        k=3.,
        m1=1.,
        m2=1.,
        m3=1.,
        spring_length=3.,
        dt=0.05,
        scale=6.,
        simulate_all=False,
):
    params = {
        'dt': dt,
        'max_time_index': MAX_TIME_INDEX,
        'block_count': 3,
    }
    initial_state = {}
    set_block_params(0, params, mass=m1)
    set_block_params(1, params, mass=m2)
    set_block_params(2, params, mass=m3)
    set_block_state(0, initial_state, position=initial_x1, velocity=initial_x2)
    set_block_state(1, initial_state, position=initial_y1, velocity=initial_y2)
    set_block_state(2, initial_state, position=initial_z1, velocity=initial_z2)
    set_spring_params(0, params, strength=b, stiffness=k, eq_length=spring_length)
    set_spring_params(1, params, strength=b, stiffness=k, eq_length=spring_length)
    
    states = simulate(params, initial_state)
    
    renderer_widget.render_func = lambda: show_render(params, states, scale/100.)

    fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15,8))
    draw_world(ax0, params, states[time_index], scale/100.)